In [1]:
import coniferest as cf
from coniferest.session.callback import (
    TerminateAfter, viewer_decision_callback)
from coniferest.session import Session
from coniferest.aadforest import AADForest
from coniferest.pineforest import PineForest
from coniferest.label import Label

import numpy as np
import pandas as pd
import time

In [3]:
oid_filename = '/media/snad/data/features/dr3/oid_683.dat'
feature_filename = '/media/snad/data/features/dr3/feature_683.dat'

oid     = np.array(np.memmap(oid_filename, mode='c', dtype=np.uint64))
feature = np.array(np.memmap(feature_filename, mode='c', dtype=np.float32).reshape(oid.shape[0], -1))


model = PineForest(
    # Number of trees to use for predictions
    n_trees=256,
    # Number of new tree to grow for each decision
    n_spare_trees=768,
    # Fix random seed for reproducibility
    random_seed=42,
)


s = Session(data=feature,
    metadata=oid,
    model=model,
    decision_callback=viewer_decision_callback,
    on_decision_callbacks=[
        TerminateAfter(3),
    ],
)


t = time.monotonic()
s.run()
t = (time.monotonic() - t) / 60
print(f'Session takes {(t / 60):.0f}h. {t:.0f}m.')

Check https://ztf.snad.space/view/683205100016505 for details
Is 683205100016505 anomaly? [y/N]:

  N


Check https://ztf.snad.space/view/683211400012356 for details
Is 683211400012356 anomaly? [y/N]:

  y


Check https://ztf.snad.space/view/683215100009351 for details
Is 683215100009351 anomaly? [y/N]:

  N


Session takes 0h. 0m.


In [ ]:
# Save answers to csv
decisions_filename = "decisions.csv"

with open(decisions_filename, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Object", "Decision"])
    for idx, label in s.known_labels.items():
        writer.writerow([oid[idx], label.name]) 